In [1]:
//https://dataplatform.ibm.com/exchange/public/entry/view/38feb8757afdb2c3c28a0dda489c2279

val link_telco = "https://raw.githubusercontent.com/AlgorithmDemo/SampleData/master/telco.csv"

import sys.process._
import java.net.URL
import java.io.File
new URL(link_telco) #> new File("telco.csv") !!

val link_telco_Feb = "https://raw.githubusercontent.com/AlgorithmDemo/SampleData/master/telco_Feb.csv"

import sys.process._
import java.net.URL
import java.io.File
new URL(link_telco_Feb) #> new File("telco_Feb.csv") !!

In [2]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

val dfTelco = sqlContext.
    read.
    format("com.databricks.spark.csv").
    option("header", "true").
    option("inferschema", "true").
    load("telco.csv")

dfTelco.show(1)

Waiting for a Spark session to start...

+------+------+---+-------+-------+------+---+------+------+------+------+--------+-----+--------+--------+-------+-------+--------+-------+-------+-------+-------+--------+-------+-------+--------+-----+-----+--------+------+--------+-------+------+-----+----------------+-------+-------+----------------+-------+----------------+-------+-----+
|region|tenure|age|marital|address|income| ed|employ|retire|gender|reside|tollfree|equip|callcard|wireless|longmon|tollmon|equipmon|cardmon|wiremon|longten|tollten|equipten|cardten|wireten|multline|voice|pager|internet|callid|callwait|forward|confer|ebill|         loglong|logtoll|logequi|         logcard|logwire|           lninc|custcat|churn|
+------+------+---+-------+-------+------+---+------+------+------+------+--------+-----+--------+--------+-------+-------+--------+-------+-------+-------+-------+--------+-------+-------+--------+-----+-----+--------+------+--------+-------+------+-----+----------------+-------+-------+----------------+--

In [3]:
dfTelco.printSchema

root
 |-- region: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital: integer (nullable = true)
 |-- address: integer (nullable = true)
 |-- income: integer (nullable = true)
 |-- ed: integer (nullable = true)
 |-- employ: integer (nullable = true)
 |-- retire: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- reside: integer (nullable = true)
 |-- tollfree: integer (nullable = true)
 |-- equip: integer (nullable = true)
 |-- callcard: integer (nullable = true)
 |-- wireless: integer (nullable = true)
 |-- longmon: double (nullable = true)
 |-- tollmon: double (nullable = true)
 |-- equipmon: double (nullable = true)
 |-- cardmon: double (nullable = true)
 |-- wiremon: double (nullable = true)
 |-- longten: double (nullable = true)
 |-- tollten: double (nullable = true)
 |-- equipten: double (nullable = true)
 |-- cardten: double (nullable = true)
 |-- wireten: double (nullable = true)
 |-- multline: int

In [4]:
val dfTelcoFeb = sqlContext.
    read.
    format("com.databricks.spark.csv").
    option("header", "true").
    option("inferschema", "true").
    load("telco_Feb.csv")

[Stage 6:=============================>                             (1 + 1) / 2]

In [5]:
import com.ibm.spss.ml.classificationandregression.ensemble.RandomTrees
import com.ibm.spss.ml.utils.DataFrameImplicits.DataFrameEnrichImplicitsClass

val ordinal = Array("ed")
val nominal = Array("region",
     "marital",
     "retire",
     "gender",
     "tollfree",
     "equip",
     "callcard",
     "wireless","multline",
     "voice","pager","internet","callid","callwait","forward","confer",
     "ebill",
     "custcat",
     "churn"
   )
val srf = RandomTrees().setTargetField("churn").setNumTrees(10)
val srfModel = srf.fit(dfTelco.setNominalMeasure(nominal,true).setOrdinalMeasure(ordinal,true))

[Stage 24:=============================>                            (1 + 1) / 2]

In [6]:
val predResult = srfModel.transform(dfTelcoFeb)
val predResultNew = predResult.withColumn("prediction", predResult("prediction").cast("double")).
    withColumn("churn", predResult("churn").cast("double"))

In [7]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
val evaluator = new MulticlassClassificationEvaluator().setLabelCol("churn").setMetricName("accuracy")
val acc_result = evaluator.evaluate(predResultNew)
println(s"acc_result:$acc_result")

acc_result:0.942


In [9]:
import com.ibm.spss.scala.ModelViewer
kernel.magics.html(ModelViewer.toHTML(pc,srfModel))

Name: Compile Error
Message: <console>:55: error: not found: value pc
       kernel.magics.html(ModelViewer.toHTML(pc,srfModel))
                                             ^
StackTrace: 

In [10]:
import java.io.{File, PrintWriter}

srfModel.toPMML("randomTrees_pmml.xml")
val statXML = srfModel.statXML()
new PrintWriter("StatXML.xml") {
      write(statXML)
      close
}

$anon$1@379ddb89